# Implement an Evolutionary Programming algorithm to solve for the Ackley Function:

$$min f(x) = -20exp(-0.2\sqrt{(\frac{1}{n})*\sum_{i=1}^{n}{x^2_i}})- exp(\frac{1}{n}*\sum_{i=1}^{n}{cos(2πx_i)}) + 20 + e$$

Which domain is: $$|x_i|≤30$$

and the local minima is on $$x_i = 0$$ and $$f(\overrightarrow{x})=0.0$$


### Considerations:

* Look aftwer having the decision variables within the specified range. If it happens, implement a mechanism for adjustment or recalculation of within the range. Explain why the mechanism was chose.

**Input:**

1. First line will contain a single number of the n-variables for the function
2. Second line will contain the population size ($\mu$) and the maximum number of generations (G) separated by a space
3. Third line will contain the value of $\alpha$ and $\epsilon$ separated by a space.

*Example:*

2

100 200

2.0 0.0001

**Output:**

A single line containing the solution x and the function evaluated with the solution separated by a space. 


*Example:*

[7.48188924e-08, -6.67209514e-07]

f(x) = 0.000



In [349]:
import math
import numpy as np
from numpy.random import normal, uniform
from statistics import stdev, mean, median
import pandas as pd

In [335]:
import decimal
decimal.getcontext().prec = 100

In [336]:
def f(x):
    n = len(x)
    #x = np.array([round(i,7) for i in x])
    sumation_1 = sum([i**2 for i in x])
    sumation_2 = sum([math.cos(2*math.pi*i) for i in x])
    term1 = 20*(math.e**(-0.2*math.sqrt(sumation_1/n)))
    term2 = math.e**(sumation_2/n)
    return (20 + math.e - term1 - math.e**sumation_2)

def fitness(x):
    return -f(x)

## Initial population:
We need to generate the decision variables first, they must be within the interval [-30,30]. with a uniform distribution.

In [337]:
#The following function is based on the function seen in class: "getInitialPopulation"

def InitialPopulation(mu,n):
    #Input: amount of individuals and the number of variables
    population = []
    
    for i in range(mu):
        variables = uniform(-30.0,30.0,n)
        sigmas = uniform(0,1,n)
        parents = [[variables,sigmas], fitness(variables)]
        population.append(parents)
    return population

In [338]:
i0 = InitialPopulation(100,2)
i0[0] #First individual

[[array([-5.8600292 , -1.67252201]), array([0.04775515, 0.00680259])],
 -13.085468989767651]

## Mutation:
If variables lie outside the limits, the easiest way of handeling is adjusting that variable to the limit of the interval whether it happened to be on the left of right limit. Still, trying to keep the uniformity, it is better to just eliminate the variable and create another until it is within the interval. This last option would take more execution time, so another way is **adding or substracting the variance value of the variables to zero in order to replace the outlier.**

In [340]:
def mutation(individual, n, epsilon,alpha):
    sigmas = individual[0]
    variables = individual[1]
    sigma_mutated = sigmas*(1+(alpha*normal(0,1,n)))
    sigma_mutated[sigma_mutated<epsilon]=epsilon #--->to actually have a change between generations, make sure there is a significant mutation variable
    
    variables_mutated = variables+(sigma_mutated*normal(0,1,n))
    
    variance = stdev(variables)**2
    check = []
    for i in range(n):
        if variables_mutated[i]>30.0:
            check.append(30.0)
        elif variables_mutated[i]< -30.0:
            check.append(-30.0)
        else:
            check.append(variables_mutated[i])
            
    individual = [[np.array(check), sigma_mutated], fitness(check)]

    return individual

In [341]:
mut = mutation(i0[0][0],2,0.0001,2.0)

In [342]:
mut,i0[0]

([[array([0.04786539, 0.00690356]), array([0.0001, 0.0001])],
  4.203492711343148],
 [[array([-5.8600292 , -1.67252201]), array([0.04775515, 0.00680259])],
  -13.085468989767651])

From the above we see that our mutations are giving good results at the fitness function. So the next step is to put it all together.  

In [343]:
def EP(mu,G,epsilon,alpha, n):
    parents = InitialPopulation(mu,n)
    
    for t in range(G):
        offsprings = []
        
        for parent in parents:
            offspring = mutation(parent[0],n,epsilon,alpha)
            offsprings.append(offspring)
        
        offpsrings = offsprings.sort(key=lambda individual: individual[1])
        
        #Selection (µ+µ)
        offsprings = offsprings[:mu]
        parents = offsprings.copy()
        
    best_individual = parents[-1]
    xbest = best_individual[0][0]
    f_best = f(xbest)
    return xbest,f_best

In [344]:
EP(100,200,0.0001,2.0,2)

(array([-9.70375860e-05, -3.67018648e-06]), -4.670498237003683)

# Statistical analysis:

Let the user to execute the program M times your algorithm for the Ackley function, report:

1. Best solution of the M exectuions
2. Worst solution of the M executions
3. Solution of the median considering the M executions.
4. Mean of the function considering the M executions.
5. Standard deviation of the functrion considering the M executions. 

Use the results for: 5, 10 and 20 variables and the parameters have to change also. 

In [371]:
def stats(M, mu, G, epsilon, alpha, n):
    f_list = []
    dic = {}
    for i in range(M):
        x,f = EP(mu,G,epsilon,alpha,n)
        f_list.append(f)
        dic[f] = x
        
    best_solution = sorted(dic)[0]
    best_x = dic[best_solution]
    
    worst_solution = sorted(dic)[-1]
    worst_x = dic[worst_solution]
    
    med_key = math.ceil(median([i for i in range(M)]))
    median_solution = f_list[med_key]
    median_x = dic[median_solution]
    
    mean_f = mean(f_list)
    standard_f = stdev(f_list)
    
    print("Best solution of f: ", best_solution, " with variables: ", best_x)
    print("Worst solution of f: ", worst_solution, " with variables: ", worst_x)
    print("Solution of the median: ", median_solution, " with variables: ", median_x)
    print("Mean of function found: ", mean_f)
    print("Standard deviation of function value: ", standard_f)
    return [best_solution, best_x, worst_solution, worst_x, median_solution, median_x, mean_f, standard_f]

In [372]:
Data = []

In [384]:
if __name__ == "__main__":
    def Read(filename):
        file1 = open(filename, 'r')
        Lines = file1.readlines()
        n = int(Lines[0])
        mu,G = map(int,Lines[1].split()) #Temperature
        alpha,epsilon = map(float,Lines[2].split()) #Amount of objects
        return n,mu,G,alpha,epsilon
    
    filename = str(input("Filename: "))
    M = int(input("M executions: "))
    if filename!="":
        n,mu,G,alpha,epsilon = Read(filename)
    else:
        n = int(input("Number of variables: "))
        mu, G = map(int, input("Size of population and Generations: ").split())
        alpha, epsilon = map(float, input("Alpha and Epsilon: ").split())
    
    arr = stats(M, mu, G, epsilon, alpha, n) 
    arr.append(M)
    arr.append(mu)
    arr.append(G)
    arr.append(alpha)
    arr.append(epsilon)
    Data.append(arr)

Filename: 
M executions: 15
Number of variables: 20
Size of population and Generations: 100 200
Alpha and Epsilon: 3.2 0.001
Best solution of f:  -470047569.1882558  with variables:  [-1.45732411e-03  5.43645729e-03  1.41656650e-03 -2.85116687e-04
 -9.04147246e-03  2.10023535e-02  1.07882513e-02  9.10693618e-04
  9.90167086e-01 -3.90212529e-04  6.10829092e-04 -1.30167927e-02
  1.57910252e-03  4.84170688e-03 -1.42959519e-03  2.69815341e-03
  3.72052927e-04 -5.49526988e-03  9.19251897e-04  2.44701268e-02]
Worst solution of f:  -298491212.5305287  with variables:  [ 2.79541951e+01  1.08700247e-03  1.25180081e-03 -7.33590539e-04
  4.00114204e-03  3.00000000e+01  1.33151654e-03  5.59905017e-05
  5.07695806e-02  2.41419522e-03 -8.12772188e-03 -7.70407189e-04
  3.00000000e+01  1.85775975e-02  2.87347124e-03  9.76169129e-04
  1.32994203e-03  1.40301863e-02  9.82699637e-03  1.43467662e-01]
Solution of the median:  -466466109.9529411  with variables:  [ 2.30454605e-03  2.54150395e-03  9.17825967

In [385]:
df1 = pd.DataFrame(Data,columns=['best_solution','best_x','worst_solution','worst_x','median_solution','median_x','mean_f','standard_f',"M-reps",'Mu','G','Alpha',"Epsilon"])
df1

,best_solution,best_x,worst_solution,worst_x,median_solution,median_x,mean_f,standard_f,M-reps,Mu,G,Alpha,Epsilon
0,-1.456554e+02,"[0.0009899970983623231, 0.0014371496812474445,...",-1.452849e+02,"[0.001006174126679851, 0.0023876644168615143, ...",-1.455988e+02,"[0.0005447234835840222, 0.003044341077559095, ...",-1.454908e+02,1.515019e-01,10,20,50,2.0,0.00100
1,-1.456948e+02,"[-5.752481809252877e-07, 2.1996156628133265e-0...",-1.456945e+02,"[-0.00015940087458750975, 4.100730159320735e-0...",-1.456948e+02,"[-1.0218329722315909e-05, 1.9246341521088267e-...",-1.456947e+02,1.344114e-04,5,50,100,2.5,0.00001
2,-1.456809e+02,"[0.00013508317358482732, 0.0010390044523893285...",-1.455724e+02,"[0.0017994253350008698, 8.097935588546499e-05,...",-1.456426e+02,"[0.003024700577273862, -0.0014517070348560239,...",-1.456435e+02,3.271959e-02,15,100,200,3.2,0.00100
3,-2.195543e+04,"[0.004035538193238391, 0.009118026842415373, 0...",-1.922372e+04,"[-0.002246324234779851, -0.002056133506204825,...",-2.140286e+04,"[0.0011052565356939202, 0.003613215446029352, ...",-2.107293e+04,1.045615e+03,10,20,50,2.0,0.00100
4,-2.202374e+04,"[-2.0450737679425854e-07, 1.0828866705421643e-...",-2.202361e+04,"[-6.140624963651856e-07, 0.0001801539849905539...",-2.202361e+04,"[5.5062324160464985e-06, 3.244035974166718e-05...",-2.202369e+04,7.034526e-02,5,50,100,2.5,0.00001
5,-2.199350e+04,"[0.006190040959546645, 0.001133910697536748, -...",-2.084512e+04,"[0.04633742821385295, 0.005579372413287241, -0...",-2.188862e+04,"[-0.007858476241628661, 0.007659488188565489, ...",-2.176883e+04,3.016473e+02,15,100,200,3.2,0.00100
6,-4.702182e+08,"[0.002067617024495502, 0.0013851571527947497, ...",-6.799091e+07,"[0.005824807942495885, 0.011548318936872892, 0...",-6.799091e+07,"[0.005824807942495885, 0.011548318936872892, 0...",-3.085858e+08,1.553902e+08,10,20,50,2.0,0.00100
7,-4.851628e+08,"[0.0003282539624477001, 0.00025466549563399446...",-4.849317e+08,"[-3.133537874791331e-05, 6.0033351188217165e-0...",-4.849995e+08,"[0.00014929659994121486, -1.115604777917556e-0...",-4.850710e+08,1.004931e+05,5,50,100,2.5,0.00001
8,-4.700476e+08,"[-0.0014573241075883613, 0.005436457292151347,...",-2.984912e+08,"[27.954195061064297, 0.001087002473714109, 0.0...",-4.664661e+08,"[0.00230454605001168, 0.0025415039454973407, 0...",-4.253846e+08,4.991497e+07,15,100,200,3.2,0.00100
